<img src="../img/Signet_FNW_1.svg" alt="OVGU_FNW_Logo" width="300" align="right">

# 2.9. Geometrical optics: Refection &amp; refraction

Nowadays we know that light has particle and wave properties, a concept known as *wave-particle duality*. 
However, when light interacts with objects much larger than its tiny wavelength, its wave nature becomes less significant. 
In such scenarios, we can effectively approximate light as traveling in straight lines called rays. 
Geometric optics is this simplified model that allows us to understand *reflection &amp; refraction* as well as (in the next chapter) lenses and optical instruments.